In [7]:
import scipy.io
from os import listdir
from os.path import isfile, join

In [8]:
mat = scipy.io.loadmat('../data/dataoncosalud/breast/bmode/08-31-49_m1.mat')

In [9]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Dec  6 03:08:19 2024',
 '__version__': '1.0',
 '__globals__': [],
 'a_0': array([[20.24993123]]),
 'b_0': array([[42.83232499]]),
 'comp_env': array([[209.42748786, 236.57626496, 215.16729021, ..., 221.62503944,
         194.55784765, 216.24853992],
        [230.13171686, 248.84047216, 228.33119469, ..., 238.06924195,
         222.5574307 , 226.12456362],
        [235.2343399 , 254.52228596, 233.98879222, ..., 241.45917672,
         223.49559007, 231.01001231],
        ...,
        [220.46236286, 229.21405365, 221.01640797, ..., 213.64778604,
         194.73944956, 196.30144159],
        [214.09517788, 230.12408597, 216.19055689, ..., 214.75497812,
         185.68101247, 203.26529017],
        [223.94658685, 233.64443504, 225.27876057, ..., 214.74635339,
         196.40115122, 202.54628962]])}

In [15]:
mat["comp_env"][:200,:].shape

(200, 256)

In [10]:
# pathbreast ='../data/dataoncosalud/breast/bmode/'
# pathganglio='../data/dataoncosalud/ganglio/bmode/'

# fnamesbreast = [pathbreast+f for f in listdir(pathbreast) if isfile(join(pathbreast, f))]
# fnamesganglio = [pathganglio+f for f in listdir(pathganglio) if isfile(join(pathganglio, f))]

In [11]:
# from sklearn.model_selection import train_test_split
# pathslst=fnamesganglio+fnamesbreast
# train_data, temp_data = train_test_split(pathslst, test_size=0.3, random_state=42)

# # Now split the remaining data into validation (50%) and test (50%)
# val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# # Output
# print("Training Data:", train_data)
# print("Validation Data:", val_data)
# print("Test Data:", test_data)

# data_splits={
#     'train_files': train_data,
#     'val_files'  : val_data,
#     'test_files' : test_data
# }

In [12]:
# import pickle

# with open('data_splits.pkl', 'wb') as handle:
#     pickle.dump(data_splits, handle)